[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sujee/mongodb-atlas-vector-search/blob/main/lab-4-rag/rag-10k-a-populate-embeddings-open.ipynb)

#  RAG-10k - Populate data with LOCAL embeddings

## Overview

Here is overall RAG pipeline.  In this notebook we will do steps 1 & 2.

This notebook showcases how to use **LOCAL EMBEDDING MODELS** to create embeddings.

We will do the following:

- 👉 Step-1: Load and process PDF documents
- 👉 Step-2: Use LOCAL embedding models to calculate embeddings for PDF documents
- 👉 Upload them into Atlas


![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/rag-overview-2.png)

### What you need to run this notebook

- a (free) MongoDB Atlas Account
- and connection credentials

### The Stack

- Langugage : Python
- Vector database: Atlas
- Embedding Model: Local embedding models from HuggingFace


### How to run

This notebook can be run on Google Colab and stand alone python development environments.  

**GPU recommended**

Click here to run on colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sujee/mongodb-atlas-vector-search/blob/main/lab-4-rag/rag-10k-a-populate-embeddings-open.ipynb)


References


## Step-1: Setup Atlas

We will need to have Atlas setup.

Follow [instructions here](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/lab-1-atlas-setup/setup-atlas.md)


## Step-2: Configuration

We will setup some common configurations here

In [1]:
# We will keep all global variables in an object to not pollute the global namespace.
class MyConfig(object):
    pass

MY_CONFIG = MyConfig()

## Atlas settings
MY_CONFIG.DB_NAME = 'rag1'
MY_CONFIG.COLLECTION_NAME = '10k_local'
MY_CONFIG.EMBEDDING_ATTRIBUTE = 'embedding_local'
MY_CONFIG.INDEX_NAME = 'idx_embedding_local'

## Embedding settings
## Option 1 : small model - about 133 MB size
## Option 2 : large model - about 1.34 GB
## See Step-12 for more details

MY_CONFIG.EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"

## Step-3: Load Configuration

We need to configure the following
- Atlas connection credentials

### Option 3A - If running on Colab

- Click on 'Colab secrets' icon (🔑) on left pane, and crate the following secret.
   - `ATLAS_URI`
-  Make sure the `notebook access` button is checked on for all secrets
- See screenshot below for example

<!-- ![](../images/colab-secret-2.png) -->

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/colab-secret-1.png)


### Option 3B - If running on local python environment

- setup your local python env following this [setup guide](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/setup-python-env.md)
- Create a file named `.env` in the same location as notebook
- And add the following settings

```text
ATLAS_URI=mongodb+srv://<username>:<password>@sandbox.....
```


## Step-4: Determine Runtime Environment

This code will figure out if we are running on Google Colab environment or local environment.  We use it to install relevant packages later.

In [2]:
# are we running in Colab?
import os

if os.getenv("COLAB_RELEASE_TAG"):
    print("Running in Colab")
    MY_CONFIG.RUNNING_IN_COLAB = True
else:
    print("NOT running in Colab")
    MY_CONFIG.RUNNING_IN_COLAB = False

Running in Colab


## Step-5: Install dependencies (if necessary)

We will install required libraries in cloud environments like Google Colab.  For local environments, we assume the dependencies are already setup.

In [3]:
if MY_CONFIG.RUNNING_IN_COLAB:
    !pip install \
                openai \
                pymongo==4.6.2 \
                llama-index \
                llama-index-embeddings-huggingface \
                llama-index-embeddings-openai \
                llama-index-vector-stores-mongodb \
                transformers==4.38.2 \
                sentence_transformers==2.5.1 \
                torch==2.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━

## Step-6: Basic Setup

### 6.1 - Check if we have GPU

In [4]:
## Check if GPU is enabled
import os
import torch

## To disable GPU and experiment, uncomment the following line
## Normally, you would want to use GPU, if one is available.
# os.environ["CUDA_VISIBLE_DEVICES"]=""

print ("using CUDA/GPU: ", torch.cuda.is_available())

for i in range(torch.cuda.device_count()):
   print("device ", i , torch.cuda.get_device_properties(i).name)

using CUDA/GPU:  True
device  0 Tesla T4


### 6.2 - Logging

In [5]:
## Setup logging.  To see more loging set the level to DEBUG

import sys
import logging

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, level=logging.WARN)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Step-7: Load Configurations

In [6]:
## Load settings based on where we are running
##  - if runninning on google Colab, load from secrets
##  - if running locally use dotenv

if MY_CONFIG.RUNNING_IN_COLAB:
    from google.colab import userdata
    MY_CONFIG.ATLAS_URI = userdata.get('ATLAS_URI')
    # MY_CONFIG.OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
else:
    import os, sys
    from dotenv import find_dotenv, dotenv_values

    this_dir = os.path.abspath('')
    parent_dir = os.path.dirname(this_dir)
    sys.path.append (os.path.abspath (parent_dir))

    config = dotenv_values(find_dotenv())
    # debug
    # print (config)
    MY_CONFIG.ATLAS_URI = config.get('ATLAS_URI')
## --- end load config

## If you just want to quickly set the config manually, you can do so here.
# MY_CONFIG.ATLAS_URI = ''

if  MY_CONFIG.ATLAS_URI:
    print ("✅ config ATLAS_URI found")
else:
    raise Exception ("'❌ ATLAS_URI' is not set.  Please set it above to continue...")


✅ config ATLAS_URI found


## Step-8 : Get Data Files (if needed)

We are going to be using 10K filings - these are financial documents filed by US public companies to SEC (Securities and Exchange Commission).  You can read about them [here](https://www.investor.gov/introduction-investing/investing-basics/glossary/form-10-k)

We have two 10k documents from Lyft and Uber

Don't think it is just 2 documents.  Each PDF documnet is about 200+ pages long.  So these are serious PDF documents.

Let's get these data files.

In [7]:
import os

# ------- begin -------
def download_data_file (remote_file, local_file):
     if  os.path.exists (local_file):
         print (f"✅ Local data files exists : {local_file}")
     else:
        !wget -O {local_file}  {remote_file}
        print (f"✅ Downloaded data file : {local_file}")
#-------- end -------

# figure out data dir
if MY_CONFIG.RUNNING_IN_COLAB:
    MY_CONFIG.DATA_DIR = "data/10k"
else:
    MY_CONFIG.DATA_DIR = "../data/10k"

if not os.path.exists (MY_CONFIG.DATA_DIR):
  !mkdir -p {MY_CONFIG.DATA_DIR}

download_data_file ('https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/data/10k/lyft_2021.pdf',
                    os.path.join (MY_CONFIG.DATA_DIR, 'lyft_2021.pdf'))

download_data_file ('https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/data/10k/uber_2021.pdf',
                    os.path.join (MY_CONFIG.DATA_DIR, 'uber_2021.pdf'))

--2024-03-22 20:32:04--  https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1440303 (1.4M) [application/octet-stream]
Saving to: ‘data/10k/lyft_2021.pdf’

data/10k/lyft_2021. 100%[===================>]   1.37M  6.85MB/s    in 0.2s    

2024-03-22 20:32:04 (6.85 MB/s) - ‘data/10k/lyft_2021.pdf’ saved [1440303/1440303]

✅ Downloaded data file : data/10k/lyft_2021.pdf
--2024-03-22 20:32:04--  https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.c

## Step-9: Inspect the PDF files

The will be in the following directory


```text
data/10k/
├── lyft_2021.pdf
└── uber_2021.pdf
```


## Step-10: Initialize Atlas Client

If this step fails, make sure 'connect from anywhere' is enabled on your Atlas network configuration

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/atlas-connect-2.png)

In [8]:
import pymongo

mongodb_client = pymongo.MongoClient(MY_CONFIG.ATLAS_URI)
print ('✅ Connected to Atlas instance!')

✅ Connected to Atlas instance!


## (Optional) Step-11: Clear out the collection

For a fresh start!

In [9]:
## if a clean start is required, you can use the following code to clear out old data
## quick fix for a bug not updating a collection (TODO: investigate)

database = mongodb_client[MY_CONFIG.DB_NAME]
collection = database [MY_CONFIG.COLLECTION_NAME]

doc_count = collection.count_documents (filter = {})
print (f"Document count in collection '{MY_CONFIG.DB_NAME}.{MY_CONFIG.COLLECTION_NAME}' before delete : {doc_count:,}")

result = collection.delete_many(filter= {})
print (f"Deleted docs : {result.deleted_count}")

Document count in collection 'rag1.10k_local' before delete : 755
Deleted docs : 755


## Step-12: Calculate Embeddings

We are going to be using embedding models that are open source

### Local Embeddings

Here are a select models for comparison.  Taken from leaderboard : https://huggingface.co/spaces/mteb/leaderboard

| model name                              | overall score | model size | model params | embedding length | License  | url                                                            |
|-----------------------------------------|---------------|------------|--------------|------------------|----------|----------------------------------------------------------------|
| BAAI/bge-large-en-v1.5                  | 64.x          | 1.34 GB    | 335 M        | 1024             | MIT      | https://huggingface.co/BAAI/bge-large-en-v1.5                  |
| BAAI/bge-small-en-v1.5                  | 62.x          | 133 MB     | 33.5 M       | 384              | MIT      | https://huggingface.co/BAAI/bge-small-en-v1.5                  |
| sentence-transformers/all-mpnet-base-v2 | 57.8          | 438 MB     |              | 768              | Apache 2 | https://huggingface.co/sentence-transformers/all-mpnet-base-v2 |
| sentence-transformers/all-MiniLM-L12-v2 | 56.x          | 134 MB     |              | 384              | Apache 2 | https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2 |
| sentence-transformers/all-MiniLM-L6-v2  | 56.x          | 91 MB      |              | 384              | Apache 2 | https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2  |

In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

## Step-13: Connect Illama-Index and MongoDB Atlas

Let's define MongoDB Atlas as our vector storage. This is critical to stored indexed data and then query

In [11]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import StorageContext


vector_store = MongoDBAtlasVectorSearch(mongodb_client = mongodb_client,
                                        db_name = MY_CONFIG.DB_NAME,
                                        collection_name = MY_CONFIG.COLLECTION_NAME,
                                        index_name  = MY_CONFIG.INDEX_NAME,
                                        embedding_key = MY_CONFIG.EMBEDDING_ATTRIBUTE,
                                        ## the following columns are set to default values
                                       # text_key = 'text', metadata_= 'metadata',
                                 )
storage_context = StorageContext.from_defaults(vector_store=vector_store)

## Step-14: Read PDF Documents

Ilmaa-index has very handy `SimpleDirectoryReader` that can read single files / multiple files / or entire directory content

In [12]:
%%time

from llama_index.core import SimpleDirectoryReader


## This reads one doc
# docs = SimpleDirectoryReader(
#     input_files=["./data/10k/uber_2021.pdf"]
# ).load_data()

## here we read entire directory content
docs = SimpleDirectoryReader(
        input_dir=MY_CONFIG.DATA_DIR
).load_data()

print (f"Loaded {len(docs)} chunks from '{MY_CONFIG.DATA_DIR}'")

Loaded 545 chunks from 'data/10k'
CPU times: user 18.3 s, sys: 75 ms, total: 18.4 s
Wall time: 18.3 s


## Step-15: Index the docs and Store them into MongoDB Atlas

When we execute the code below, the following will happen

- documents are indexed
- embeddings are created for text
- the document (text, embeddings) are stored in our Vector Storage (MongoDB Atlas in this case)

**Note: This might take a few minutes**

Quick benchmark on Google Colab (GPU will make a big difference!)

| Env      | Runtime      |   |
|----------|--------------|---|
| CPU      | 3 m 30 secs |   |
| GPU T4   | 20 secs      |   |
| GPU V100 | 15 secs      |   |
| GPU A100 | 12 secs      |   |

In [13]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context
)


CPU times: user 15.7 s, sys: 305 ms, total: 16 s
Wall time: 19.6 s


## Step-16: View Created Documents in Atlas

- Go to your Atlas dashboard
- Select 'browse collections'
- Select database: `rag1`  and collection `10k`
- Click around to see some sample data inserted
- You will see `text` attribute having text data
- `embeddings` are populated too
- expand the `meta` attribute.  This is automatically populated for us by llama-index

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/10k-documents-1.png)

## Step-17: Setup Vector Index

Before we do vector search, we need to define an embedding index

You can look at steps here [setup-atlas-index.md](setup-atlas-index.md)

Here are the details:

- database : **`rag1`**
- Collection: **`10k_local`**
- index_name = **`idx_embedding_local`**

index defitintion json

```json
{
  "fields": [
    {
      "type": "vector",
      "path": "embedding_local",
      "numDimensions": 384,
      "similarity": "dotProduct"
    }
  ]
}
```

The similarity function can be  one of
- "euclidean"
- "cosine"
- "dotProduct"


### Follow these steps to setup index


![missing image](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/atlas-index-2.png)

![missing image](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/atlas-index-rag-local-1b.png)

![missing image](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/atlas-index-rag-local-2b.png)



## We are done

Now the data is populated and ready to be queried.

Let's go to the next lab: query